# Python Tools for Genome-wide Gene-level Association

## Steven D. Shaw

If you would like to use this code, please feel free to do so, and give credit if it is due. I am happy to answer any questions regarding these tools, or, if you would like to work together on a project related to gene-level association, please contact me.

Email: shawsd@umich.edu

If you happen to notice any errors, please do let me know - we all make mistakes and we can always help eachother overcome and learn from our mistakes :)

### IMPORT PACKAGES

In [ ]:
import pandas as pd
import numpy as np
from pandas_plink import read_plink1_bin
from os.path import join
from os import path
from pandas_plink import get_data_folder
from pandas_plink import read_plink
import os
from os import listdir
from os.path import isfile, join
from datetime import datetime

import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.api import add_constant
from sklearn import decomposition

import warnings
warnings.filterwarnings('ignore')

import time

from factor_analyzer import FactorAnalyzer
from sklearn.preprocessing import StandardScaler
import datetime

## DATA LOADING & MERGING

## Genetic Data

In [ ]:
# Load in genetic data
    # QC PLINK vars: --mind 0.05 --geno 0.01 --maf 0.01
    # Rename SNPs: --update-map SNP_kgpID2rsID.txt --update-name

# Insert name of genetic data file - should be precleaned in PLINK
gene_data_filename = ''
    
(bim, fam, bed) = read_plink(gene_data_filename, verbose=False)
bim.set_index('chrom')
fam.set_index('i')
bim = bim.sort_values(by=['chrom', 'pos'])

bim.to_csv("bim.csv")

########

# Provide a genome gene set list (E.g., 'glist-hg19.txt')
gene_set_filename = ''

# List of genes in the human genome, and their location (chromosome, start, and end)
hg = pd.read_csv(gene_set_filename, 
                 sep=' ', 
                 header=None, 
                 names=["chromosome", "Start", "End", "gene"])

# Add column that is current position just in case for joining afterwards
hg['row_num'] = np.arange(len(hg))

hg = hg.sort_values(by=['Start', 'End'])
hg

## Survey Data

In [ ]:
# Insert name of survey data file - this file should include: dependent variables, age, 
# population stratification information from genome-wide PCA (labelled EV1, EV2, etc.), and 
# IDs that match genetic data
survey_data_filename = ''
    
DATA_survey = pd.read_csv(survey_data_filename)

## MAP SNPs TO GENES

In [ ]:
# Reorganize the genetic data (bim) to determine every gene that each SNP is located on
# Use gene location information in hg file (e.g., glist-hg19.txt) to traverse through chromosomes, sequentially,
# gene-by-gene and identify which SNPs are in each gene

hg.set_index('chromosome')

snp_to_gene_mapping = []
for chrome_i in ['1', '2', '3', '4', '5', '6', '7', 
               '8', '9', '10', '11', '12', '13', '14', 
               '15', '16', '17', '18', '19', '20', '21', '22']:
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time =", current_time)
    print("chromosome: ", chrom_i)
    chrom_x = bim.query("chrom=='{0}'".format(chrom_i))
    hg_x = hg.query("chromosome=='{0}'".format(chrom_i))
    start = hg_x['Start'].min()
    end = hg_x['End'].max()
    chrom_x = chrom_x[chrom_x["pos"].between(start, end)]
    length = len(chrom_x)

    j = 0
    
    for ind, r in hg_x.iterrows():
        while chrom_x.iloc[j, 3] < r['Start']:
            j += 1
            if j >= length:
                break
        if j >= length:
                break
        while chrom_x.iloc[j, 3] <= r['End']:
            snp_to_gene_mapping.append([chrom_x.iloc[j, 0], chrom_x.iloc[j, 1], chrom_x.iloc[j, 2], 
                                        chrom_x.iloc[j, 3], chrom_x.iloc[j, 4], chrom_x.iloc[j, 5], chrom_x.iloc[j, 6], 
                                        r['Start'], r['End'], r['gene']])
            j += 1
            if j >= length:
                break
        if j >= length:
                break

In [ ]:
# Organize traversal into dataframe
bim_with_genes = pd.DataFrame(snp_to_gene_mapping, columns = ['chrom' , 'snp', 'cm', 'pos', 
                                                               'a0', 'a1', 'i','start', 'end', 'gene'])
print(bim_with_genes)

# Save this file, so you don't need to rerun above code on other computers
# bim_with_genes.to_csv("bim_with_genes.csv")

In [ ]:
## Load in bim_with_genes for EigenSNP function
#bim_with_genes = pd.read_csv('bim_with_genes.csv')
#print(bim_with_genes)

### Required Functions

#### PCA/ FA NOTES:

Takes in a df where each row is a participant and each column is a SNP. Data input should be minor allele dosage.

PCA is run once per gene in main function. Exploratory FA is run once to determine number of factors, followed by FA used for factor extraction.

Output a df where each row is a participant and each column contains the eigenvectors from the PCA or FA factors.

The number of PCs is determined by the cummulative variance required set by the 'limit' in the function (typically, greater than 90% variance explained). For FA, number of factors is calcaluate as factors with eigenvalues >1. 

#### REGRESION NOTES:

Takes in two dfs:

i_df: a df where rows are participants and columns are the independent variables.

t_df: a df where rows are participants and column is the target dependent variable (i.e., BMI)

NOTE: Both dataframes should have same participant order.

### PCA Function

In [ ]:
def PC_eigen_vals(gene_df):
    X = StandardScaler().fit_transform(gene_df)
    limit = 0.9

    #perform PCA
    pca = decomposition.PCA(n_components=len(gene_df.columns))
    pca.fit(X)
    X = pca.transform(X)

    #extract variance for each PC
    var = pca.explained_variance_ratio_

    # find num PCs
    #np.cumsum(var)

    # find num PCs
    cum_var = 0
    i = 0
    while cum_var <= limit and i < len(var):
        cum_var += var[i]
        i += 1

    # separate eigen vect columns based on limit
    X_PCs = X[:,0:i]


    # add to dictionary
    PCs = {}
    for i in range(0,X_PCs.shape[1]):
        PCs["PC{0}".format(i)] = pd.Series(X[:,i:i+1].transpose()[0])

    # convert to df
    PC_df = pd.DataFrame(PCs)

    # Returns pd df
    return PC_df

### FA Function

In [ ]:
def FA_eigen_vals(gene_df):
    X_norm = StandardScaler().fit_transform(gene_df)
    limit = 1

    #perform exploratory factor analysis to determine number of factors with eigenvalue over 1
    efa = FactorAnalyzer(rotation=None, n_factors=len(gene_df.columns))
    efa.fit(X_norm)
    ev, v = efa.get_eigenvalues()
    gene_fa = sum(ev > limit)

    #Run factor analysis with number of factors from above EFA
    gene_fa = FactorAnalyzer(rotation="promax", n_factors=gene_fa)
    gene_fa.fit(X_norm)

    #extract eigen vector columns from FA
    FA_eigen = gene_fa.fit_transform(X_norm)
    #print(FA_eigen)

    # add to dictionary
    FAs = {}
    for i in range(0,FA_eigen.shape[1]):
        FAs["FA{0}".format(i)] = pd.Series(FA_eigen[:,i:i+1].transpose()[0])

    # convert to df
    FA_df = pd.DataFrame(FAs)
    return(FA_df)

### Regression Function

In [ ]:
def OLS_mod(i_df, t_df):
    # set dep and ind variables
    X = i_df[[i for i in i_df.columns]]
    y = t_df[[i for i in t_df.columns]]

    # add constant (intercepts)
    X = sm.add_constant(X)

    # least squares model
    mod = sm.OLS(y, X).fit()
    #print(mod.summary())

    return mod

## GENE-LEVEL GENOME-WIDE ASSOCIATION FUNCTION

In [ ]:
# Provide a character list of the variables names for your dependent measures (from DATA_survey file)
DV_list = ['']

In [ ]:
Gene_results = []
EigenSNPWAS_results = []
FactanalSNPWAS_results = []

# This loop takes each gene that we observe, obtains only the genetic dosage data that for that gene 
# from the main genetic dataset, then:
    # conducts PCA and FA on SNPs within that gene, and retains eigenvectors from the number of PCs
        # that explain >90% variance (i.e., eigenSNPs) or factors with eigenvalues >1
    # merges the eigenSNPs with survey/non-genetic data for use with regression
    # runs treatment and control regression models and compares these models to get the unique effect of the gene
    # outputs the number of SNPs observed in the gene, the number of eigenSNPs, and the p-value

Gene_results = pd.DataFrame(columns=['Gene', 'SNPs', 'ss_diff', 'f_stat','p_val'])
EigenSNPWAS_results = pd.DataFrame(columns=['Gene', 'SNPs', 'EigenSNPs','PCA_ss_diff', 'PCA_f_stat','PCA_p_val'])
FactanalSNPWAS_results = pd.DataFrame(columns=['Gene', 'SNPs', 'No_factors', 'FA_ss_diff', 'FA_f_stat','FA_p_val'])

bim_with_genes.set_index('chrom')

#Get a list of chromosomes in genetic dataset
chrom_x = bim_with_genes['chrom'].unique()

#Current time
now = datetime.datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

#Start timer
start_time = time.time()

for j in chrom_x:

    time_chk_chrom_loop = time.time()

    print('Chromosome:', j)
    chrom_df = bim_with_genes.query("chrom=={0}".format(j))
    #print(chrom_df)
    #Create list of observed genes
    chrom_df.set_index('gene')
    genes_in_chrom_x = chrom_df['gene'].unique()
    #print(genes_in_chrom_x)

    #Chromosome counter so we can stop the loop, for testing
    gene_counter = 0

    #t1
    time_chk_1 = time.time()

    for i in genes_in_chrom_x:
        #print(i)

        time_chk_gene_loop = time.time()

        #Specify how many genes to run - remove to run all
    #    if gene_counter == 10:
    #        break

        #Display progress bar
        if gene_counter in list(range(1, len(genes_in_chrom_x), 50)): #Change division size for different progress indicators
            print('Gene:', gene_counter, 'of', len(genes_in_chrom_x))
            elapsed_time = time.time() - start_time
            print("Elapsed Time =", time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

        # Get the SNP data from bed file
        gene_df = chrom_df.query("gene=='{0}'".format(i))
        x = bed[gene_df.i.values, :].compute() #x contains only dosage data for SNPs in current gene

        # transpose so that data are formatted correctly (columns are SNPs, rows are participants)
        x_trans = x.transpose()
        #print(x_trans)

        # Get SNP names, create dataframe with SNP labels (as columns) for use in PCA
        snps = gene_df['snp']
        SNP_data = pd.DataFrame(x_trans, columns = snps)
        #print('with NaN:', SNP_data)

        #There are some strange columns of all NaNs, remove these first
        SNP_data.dropna(axis=0, how='all', inplace=True)

        #Replace NaNs with mean for each column
        #SNP_data.fillna(SNP_data.mean(), axis=0, inplace=True)
        SNP_data = SNP_data.apply(lambda x: x.fillna(x.mean()),axis=0)

        #finally, if any NAs remain within a row, drop that row
            #very few of these cases, but needed to keep function running (e.g., CSMD1 in chr8, has random missingness)
        #SNP_data.dropna(axis=1, how='any', inplace=True)
        #print(SNP_data.shape)
        #print('without NaN:', SNP_data)

        #Create a seperate dataset for PCA
        SNP_data_for_PC = SNP_data.copy()

        #Dataset for FA
        #Need to remove any columns (SNPs) where standard deviation = 0
        SNP_data_for_FA = SNP_data.drop(SNP_data.std()[SNP_data.std() == 0].index.values, axis=1)

        #Count number of SNPs
        SNP_count = len(SNP_data.columns)
        SNP_count_PCA = len(SNP_data_for_PC.columns)
        SNP_count_FA = len(SNP_data_for_FA.columns)
        #print(i)
        #print(SNP_count)

    #t2
        time_chk_2 = time.time()

    ####################################################
    ######RAW SNP DATA - NO PCA OR FA

    ################################
        skip_pca = 0
        skip_fa = 0

        if SNP_count_PCA > 1:
            try:
                PCA_og = PC_eigen_vals(SNP_data_for_PC)

            except:
                skip_pca = 1

        if SNP_count_FA > 1:
            try:
                FA_og = FA_eigen_vals(SNP_data_for_FA)

            except:
                skip_fa = 1
    ################################

        #Traverse DV_list
        for DV in DV_list:

            if SNP_count == 0:
                #EigenSNPWAS.append([i, 0, np.nan, np.nan])
                Gene_results = Gene_results.append({'Gene': i,
                                'SNPs': 0,
                                'ss_diff': np.nan,
                                'f_stat': np.nan,
                                'p_val': np.nan,
                                'DV': np.nan},
                                ignore_index=True)

            else:
                try:

                    if SNP_count == 1:
                        Gene_reg = SNP_data.copy()
                    else:
                        Gene_reg = SNP_data.copy()

                    # Take output of PCA - eigenSNPs - and merge with survey/non-genetic data
                    #Get ID codes for gene data - use key values (i) from query with gene name
                        #gene_df contains only the SNPs we are interested in

                    #Append gene IDs to PCA output
                    Gene_reg['IID'] = pd.to_numeric(fam['iid'], errors="coerce")

                    # new code - Add Sex variable from fam file
                    Gene_reg['Sex'] = pd.to_numeric(fam['gender'], errors="coerce")

                    #merge eigenSNPs with dataset: DATA_survey
                    Gene_reg = Gene_reg[Gene_reg['IID'].notna()]

                    DATA_survey_DV = DATA_survey[[DV, 'IID', 'Age', 'EV1', 'EV2', 'EV3', 'EV4']]

                    DATA_all_for_gene = pd.merge(DATA_survey_DV, Gene_reg, left_on='IID', right_on='IID', how='inner')

                    #Drop the IID column, as it is no longer needed
                    DATA_all_for_gene.drop('IID', axis=1, inplace=True)

                    #Remove any rows with missing values
                    DATA_all_for_gene.dropna(inplace=True)

                    DATA_DV_only = pd.DataFrame(DATA_all_for_gene[DV])

                    #Create treatment dataset
                    DATA_treatment_model = DATA_all_for_gene.drop(DV, axis=1)

                    #Create control dataset
                    DATA_control_model = DATA_all_for_gene[['Age', 'Sex', 'EV1', 'EV2', 'EV3', 'EV4']]

                    # TREATMENT Model
                    treat_mod = OLS_mod(DATA_treatment_model, DATA_DV_only)     ## controls + EigenSNPs

                    # CONTROL Model
                    cont_mod = OLS_mod(DATA_control_model, DATA_DV_only)   ## controls only

                    #Conduct ANOVA to compare TREATMENT and CONTROL models
                    aov_table = sm.stats.anova_lm(cont_mod, treat_mod, typ=1)

                    # Extract stats from model (F and p-val)
                    gene_ss_diff = aov_table['ss_diff'][1]
                    gene_fstat = aov_table['F'][1]
                    gene_pval = aov_table['Pr(>F)'][1]
                    #print('f:', gene_fstat, 'p:', gene_pval)

                    Gene_results = Gene_results.append({'Gene': i,
                                'SNPs': SNP_count_PCA,
                                'ss_diff': gene_ss_diff,
                                'f_stat': gene_fstat,
                                'p_val': gene_pval,
                                'DV': DV},
                                ignore_index=True)

                except:

                    Gene_results = Gene_results.append({'Gene': i,
                                        'SNPs': SNP_count_PCA,
                                        'ss_diff': np.nan,
                                        'f_stat': np.nan,
                                        'p_val': np.nan,
                                        'DV': DV},
                                        ignore_index=True)



    ####################################################
    ####PCA

            time_chk_3 = time.time()
            #print('before PCA:', SNP_data_for_PC)

            if SNP_count_PCA == 0:

                EigenSNPWAS_results = EigenSNPWAS_results.append({'Gene': i,
                                'SNPs': 0,
                                'EigenSNPs': np.nan,
                                'PCA_ss_diff': np.nan,
                                'PCA_f_stat': np.nan,
                                'PCA_p_val': np.nan,
                                'DV': np.nan},
                                ignore_index=True)
            else:
                try:

                    if SNP_count_PCA == 1:
                        PCA = SNP_data_for_PC.copy()
                    else:
                        #Conduct PCA - output contains dataframe with EigenSNPs (cut to >90% threshold)
                        PCA = PCA_og.copy()
                        if skip_pca == 1:
                            raise Exception

                    #Count number of EigenSNPs
                    PC_Eigen_count = len(PCA.columns)

                    # Take output of PCA - eigenSNPs - and merge with survey/non-genetic data
                    #Get ID codes for gene data - use key values (i) from query with gene name
                        #gene_df contains only the SNPs we are interested in

                    #Append gene IDs to PCA output
                    PCA['IID'] = pd.to_numeric(fam['iid'], errors="coerce")
                    # new code - Add Sex variable from fam file
                    PCA['Sex'] = pd.to_numeric(fam['gender'], errors="coerce")
                    #print('fam iid_type:', PCA['IID'].dtypes)
                    #print(PCA)

                    #merge eigenSNPs with dataset: DATA_survey
                    PCA = PCA[PCA['IID'].notna()]

                    DATA_survey_DV = DATA_survey[[DV, 'IID', 'Age', 'EV1', 'EV2', 'EV3', 'EV4']]

                    DATA_all_for_gene_PCA = pd.merge(DATA_survey_DV, PCA, left_on='IID', right_on='IID', how='inner')
                    #print(list(DATA_all_for_gene.columns)) #View columns in dataset

                    #Drop the IID column, as it is no longer needed
                    DATA_all_for_gene_PCA.drop('IID', axis=1, inplace=True)
                    #Remove any rows with missing values
                    DATA_all_for_gene_PCA.dropna(inplace=True)

                    DATA_DV_only_PCA = pd.DataFrame(DATA_all_for_gene_PCA[DV])

                    #Create treatment dataset
                    DATA_treatment_model_PCA = DATA_all_for_gene_PCA.drop(DV, axis=1)
                    #print(DATA_treatment_model)

                    #Create control dataset
                    DATA_control_model_PCA = DATA_all_for_gene_PCA[['Age', 'Sex', 'EV1', 'EV2', 'EV3', 'EV4']]
                    #print(DATA_control_model)

                    # TREATMENT Model
                    treat_mod_PCA = OLS_mod(DATA_treatment_model_PCA, DATA_DV_only_PCA)     ## controls + EigenSNPs

                    #treat_mod.summary()

                    # CONTROL Model
                    cont_mod_PCA = OLS_mod(DATA_control_model_PCA, DATA_DV_only_PCA)
                    #cont_mod.summary()

                    #Conduct ANOVA to compare TREATMENT and CONTROL models
                    aov_table_PCA = sm.stats.anova_lm(cont_mod_PCA, treat_mod_PCA, typ=1)
                    #print(aov_table)

                    # Extract stats from model (F and p-val)
                    PCA_ss_diff = aov_table_PCA['ss_diff'][1]
                    PCA_fstat = aov_table_PCA['F'][1]
                    PCA_pval = aov_table_PCA['Pr(>F)'][1]

                    #print('Gene:', i, 'SNPs:', SNP_count, 'EigenSNPs:', PC_Eigen_count, 'p_val:', gene_pval)

                    EigenSNPWAS_results = EigenSNPWAS_results.append({'Gene': i,
                                                                      'SNPs': SNP_count_PCA,
                                                                      'EigenSNPs': PC_Eigen_count,
                                                                      'PCA_ss_diff': PCA_ss_diff,
                                                                      'PCA_f_stat': PCA_fstat,
                                                                      'PCA_p_val': PCA_pval,
                                                                      'DV': DV},
                                                                      ignore_index=True)

                except:

                    EigenSNPWAS_results = EigenSNPWAS_results.append({'Gene': i,
                                                      'SNPs': SNP_count_PCA,
                                                      'EigenSNPs': np.nan,
                                                      'PCA_ss_diff': np.nan,
                                                      'PCA_f_stat': np.nan,
                                                      'PCA_p_val': np.nan,
                                                      'DV': DV},
                                                      ignore_index=True)

    ####################################################
                ####FA

            time_chk_4 = time.time()

            if SNP_count_FA == 0:
                FactanalSNPWAS_results = FactanalSNPWAS_results.append({'Gene': i,
                    'SNPs': 0,
                    'No_factors': np.nan,
                    'FA_ss_diff': np.nan,
                    'FA_f_stat': np.nan,
                    'FA_p_val': np.nan,
                    'DV': np.nan},
                    ignore_index=True)

            else:
                try:

                    if SNP_count_FA == 1:
                        FA = SNP_data_for_FA.copy()

                    else:
                        #Conduct Factor Analysis
                        FA = FA_og.copy()
                        if skip_fa == 1:
                            raise Exception

                    #Count number of EigenSNPs
                    FA_Eigen_count = len(FA.columns)
                    #print(PC_Eigen_count)

                    # Take output of PCA - eigenSNPs - and merge with survey/non-genetic data
                    #Get ID codes for gene data - use key values (i) from query with gene name
                        #gene_df contains only the SNPs we are interested in

                    #Append gene IDs to PCA output
                    FA['IID'] = pd.to_numeric(fam['iid'], errors="coerce")
                    # new code - Add Sex variable from fam file
                    FA['Sex'] = pd.to_numeric(fam['gender'], errors="coerce")
                    #print('fam iid_type:', PCA['IID'].dtypes)
                    #print(PCA)

                    #merge eigenSNPs with dataset: DATA_survey
                    FA = FA[FA['IID'].notna()]

                    DATA_survey_DV = DATA_survey[[DV, 'IID', 'Age', 'EV1', 'EV2', 'EV3', 'EV4']]

                    DATA_all_for_gene_FA = pd.merge(DATA_survey_DV, FA, left_on='IID', right_on='IID', how='inner')
                    #print(list(DATA_all_for_gene.columns)) #View columns in dataset

                    #Drop the IID column, as it is no longer needed
                    DATA_all_for_gene_FA.drop('IID', axis=1, inplace=True)
                    #Remove any rows with missing values
                    DATA_all_for_gene_FA.dropna(inplace=True)

                    DATA_DV_only_FA = pd.DataFrame(DATA_all_for_gene_FA[DV])

                    #Create treatment dataset
                    DATA_treatment_model_FA = DATA_all_for_gene_FA.drop(DV, axis=1)
                    #print(DATA_treatment_model)

                    #Create control dataset
                    DATA_control_model_FA = DATA_all_for_gene_FA[['Age', 'Sex', 'EV1', 'EV2', 'EV3', 'EV4']]
                    #print(DATA_control_model)

                    # TREATMENT Model
                    treat_mod_FA = OLS_mod(DATA_treatment_model_FA, DATA_DV_only_FA)     ## controls + EigenSNPs

                    #treat_mod.summary()

                    # CONTROL Model
                    cont_mod_FA = OLS_mod(DATA_control_model_FA, DATA_DV_only_FA)
                    #cont_mod.summary()

                    #Conduct ANOVA to compare TREATMENT and CONTROL models
                    aov_table_FA = sm.stats.anova_lm(cont_mod_FA, treat_mod_FA, typ=1)
                    #print(aov_table)

                    # Extract stats from model (F and p-val)
                    FA_ss_diff = aov_table_FA['ss_diff'][1]
                    FA_fstat = aov_table_FA['F'][1]
                    FA_pval = aov_table_FA['Pr(>F)'][1]

                    #print('Gene:', i, 'SNPs:', SNP_count, 'EigenSNPs:', PC_Eigen_count, 'p_val:', gene_pval)

                    FactanalSNPWAS_results = FactanalSNPWAS_results.append({'Gene': i,
                                                                            'SNPs': SNP_count_FA,
                                                                            'No_factors': FA_Eigen_count,
                                                                            'FA_ss_diff': FA_ss_diff,
                                                                            'FA_f_stat': FA_fstat,
                                                                            'FA_p_val': FA_pval,
                                                                            'DV': DV},
                                                                            ignore_index=True)

                except:

                    FactanalSNPWAS_results = FactanalSNPWAS_results.append({'Gene': i,
                                                            'SNPs': SNP_count_FA,
                                                            'No_factors': np.nan,
                                                            'FA_ss_diff': np.nan,
                                                            'FA_f_stat': np.nan,
                                                            'FA_p_val': np.nan,
                                                            'DV': DV},
                                                            ignore_index=True)

    #t4
        #print('near end:', SNP_data_for_PC)
        time_chk_end = time.time()
        gene_counter += 1

In [ ]:
# Output datafiles for each gene-level representation
Gene_results.to_csv()
EigenSNPWAS_results.to_csv()
FactanalSNPWAS_results.to_csv()